### Preprocessing  
Sofia, Angel

In [4]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# list of file name strings from S1 and S2
S1_files = sorted(glob.glob('../S1_Dataset/d1p*'))
S2_files = sorted(glob.glob('../S2_Dataset/d2p*'))

# TODO: when done with program, merge all data from S1 and S2 into 1 giant dataset
# for testing only using 1 file for now
file = S1_files[0]


df = pd.read_csv(file, header=None)
df.columns = [
    'time',
    'frontal accel',
    'vertical accel',
    'lateral accel',
    'antenna id',
    'rssi',
    'phase',
    'frequency',
    'activity',
]
activities = {
    1: 'sit on bed',
    2: 'sit on chair',
    3: 'lying',
    4: 'ambulating',
}
df.replace({'activity': activities}, inplace=True)

# TODO: add 3 columns activity status, frequency of activity type (time over total time), and gender 
# our target is "activity status" (inactive or active) with our independent variables being activity type, time, rssi, frontal accel, vertical accel, lateral accel, gender, frequency (of activity type)


# TODO: preprocess data and split into test and training sets using df
# normalize data 
# split 25:75 training and test
df

,time,frontal accel,vertical accel,lateral accel,antenna id,rssi,phase,frequency,activity
0,0.00,0.27203,1.00820,-0.082102,1,-63.5,2.42520,924.25,sit on bed
1,0.50,0.27203,1.00820,-0.082102,1,-63.0,4.73690,921.75,sit on bed
2,1.50,0.44791,0.91636,-0.013684,1,-63.5,3.03110,923.75,sit on bed
3,1.75,0.44791,0.91636,-0.013684,1,-63.0,2.03710,921.25,sit on bed
4,2.50,0.34238,0.96229,-0.059296,1,-63.5,5.89200,920.25,sit on bed
...,...,...,...,...,...,...,...,...,...
396,251.00,0.87003,0.46851,0.009122,4,-58.5,0.73631,921.75,sit on bed
397,251.25,0.87003,0.46851,0.009122,1,-57.5,0.70870,923.25,sit on bed
398,251.50,0.87003,0.46851,0.009122,1,-59.0,0.25157,924.75,sit on bed
399,253.25,-0.00938,0.95081,-0.116310,1,-58.5,0.35282,924.25,sit on bed


The code above wasn't working when I ran it on my computer. I'm not as experienced in jupyter notebook so maybe I'm doing something wrong? This version of the code should work if you're trying to extract the files from the zipped folder. -- Angel

In [35]:
from zipfile import ZipFile, ZipInfo
import pandas as pd

## this is the zipped folder name on my computer, idk if its the same for everyone
zipped_name = 'activity+recognition+with+healthy+older+people+using+a+batteryless+wearable+sensor.zip'

## honestly, idk what this is doing, but i feel its important stuff
z = ZipFile(zipped_name, 'r')
ls1 = z.infolist()
ls2 = [file for file in ls1 if file.file_size > 0]


In [36]:
## im going to seperate the files into four categories to make adding extra attributes easier
S1F = []
S1M = []
S2F = []
S2M = []
## this one is just to check my work
other = [] #ignore

## i feel like there was a more efficient way to do this
## but this segment of code extracts the content of the files
 # and converts them into pandas dataframes
for i in range(len(ls2)):
    temp_zip = z.extract(member=ls2[i])
    if (("S1_Dataset" in temp_zip) and ("F" in temp_zip)):
        temp_csv = pd.read_csv(temp_zip, header=None)
        S1F.append(temp_csv)
    elif (("S1_Dataset" in temp_zip) and ('M' in temp_zip) and ('.txt' not in temp_zip)):
        temp_csv = pd.read_csv(temp_zip, header=None)
        S1M.append(temp_csv)
    elif (("S2_Dataset" in temp_zip) and ("F" in temp_zip)):
        temp_csv = pd.read_csv(temp_zip, header=None)
        S2F.append(temp_csv)
    elif (("S2_Dataset" in temp_zip) and ("M" in temp_zip) and ('.txt' not in temp_zip)):
        temp_csv = pd.read_csv(temp_zip, header=None)
        S2M.append(temp_csv)
    else:
        other.append(temp_zip)

In [37]:
## all dataframes in one list, seperated into four categories
 # (based on gender and room)
all_datasets = [S1F, S1M, S2F, S2M]

features = [
    'time',
    'frontal accel',
    'vertical accel',
    'lateral accel',
    'antenna id',
    'rssi',
    'phase',
    'frequency',
    'activity',
]

activities = [
    'sitting on bed',
    'sitting on chair',
    'lying',
    'ambulating'
]

## in this segment of code, i rename the headers
new_headers = dict(enumerate(features))
new_activities = dict(enumerate(activities, start=1))
for subset in all_datasets:
    for df in subset:
        df.rename(columns = new_headers, inplace= True)
        df['activity'] = df['activity'].replace(new_activities)

## checking my work
S1F[0]

,time,frontal accel,vertical accel,lateral accel,antenna id,rssi,phase,frequency,activity
0,0.00,0.51826,0.89339,0.13456,4,-56.5,5.836800,921.75,sitting on bed
1,0.25,0.51826,0.89339,0.13456,3,-68.0,4.841200,925.75,sitting on bed
2,0.75,0.51826,0.89339,0.13456,4,-55.5,3.641700,924.25,sitting on bed
3,1.25,0.51826,0.89339,0.13456,3,-57.5,1.777900,924.75,sitting on bed
4,1.75,0.51826,0.89339,0.13456,4,-61.5,0.240830,922.75,sitting on bed
...,...,...,...,...,...,...,...,...,...
87,268.75,0.44791,-0.28938,0.94417,4,-59.0,1.455700,923.25,lying
88,273.75,0.44791,-0.28938,0.94417,4,-60.0,1.414300,923.25,lying
89,294.53,0.45964,-0.28938,0.94417,4,-63.0,0.084369,920.75,lying
90,318.50,0.58862,-0.34679,1.00120,4,-58.0,5.907400,925.25,lying


### Building + Testing Model

**Hyper param testing**  
Martin  


In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasRegressor

# TODO: add hyperparameters that we will be testing as parameters for this function (Martin)
def create_model(): 
    print("NOT IMPLEMENTED")
    # TODO: implement our nueral network structure using SGD as the optimization technique (Martin)

    # maybe use tf.keras.sequential here?
    model = 0
    return model
    
# TODO: add hyper parameters that we will be testing to param_grid (Martin)
param_grid = {
    "activation"
}

keras_model = KerasRegressor(build_fn=create_model, verbose=0)

# TODO: for GridSearchCV modify any other params that we need (scoring, cv, num_jobs, etc) (Martin)
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid)
grid_result = grid.fit(X_train_scaled, y_train)

# TODO: implement model evaluation and print/graph anything needed for visualization